# Оптимизация гиперпараметров нейросети с помощью Keras Tuner

Чтобы запускать и редактировать код, сохраните копию этого ноутбука себе (File->Save a copy in Drive...). Свою копию вы сможете изменять и запускать.

Не забудьте подключить GPU, чтобы сеть обучалась быстрее (Runtime -> Change Runtime Type -> Hardware Accelerator -> GPU).



## Гиперпараметры обучения нейронной сети

- Количество слоев нейронной сети
- Количество нейронов в каждом слое
- Функции активации, которые используются в слоях
- Тип оптимизатора при обучении нейронной сети
- Количество эпох обучения

## Установка Keras Tuner

In [1]:
pip install -U keras-tuner

     |████████████████████████████████| 133 kB 5.2 MB/s 


## Подключаем нужные пакеты

In [2]:
%tensorflow_version 2.x
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
from tensorflow.keras.preprocessing import image
from google.colab import files
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  


## Подготовка данных для обучения сети

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [4]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train / 255 
x_test = x_test / 255 
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

##Задаем функцию создания нейронной сети

In [5]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_input',
                                   min_value=128,
                                   max_value=1024,
                                   step=32),
                    input_dim=784,
                    activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(
        optimizer='SGD',
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

##Создаём тюнер

Доступные типы тюнеров: 
- RandomSearch - случайный поиск.
- Hyperband - алгоритм оптимизации на основе многорукого бандита, Li, Lisha, and Kevin Jamieson. ["Hyperband: A Novel Bandit-Based Approach to Hyperparameter Optimization."Journal of Machine Learning Research 18 (2018): 1-52](http://jmlr.org/papers/v18/16-558.html).
- BayesianOptimization - [байесовская оптимизация](https://en.wikipedia.org/wiki/Bayesian_optimization).

In [6]:
tuner = RandomSearch(
    build_model,                 # функция создания модели
    objective='val_accuracy',    # метрика, которую нужно оптимизировать - 
                                 # доля правильных ответов на проверочном наборе данных
    max_trials=10,               # максимальное количество запусков обучения 
    directory='test_directory'   # каталог, куда сохраняются обученные сети  
    )

##Запускаем подбор гиперпараметров

Пространство поиска

In [7]:
tuner.search_space_summary()

Search space summary
Default search space size: 1
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 1024, 'step': 32, 'sampling': None}


Подбор гиперпараметров

In [8]:
tuner.search(x_train,                  # Данные для обучения
             y_train,                  # Правильные ответы
             batch_size=256,           # Размер мини-выборки
             epochs=3,                 # Количество эпох обучения 
             validation_split=0.2,
             verbose=1,
             )

Trial 10 Complete [00h 00m 05s]
val_accuracy: 0.7790833115577698

Best val_accuracy So Far: 0.7918333411216736
Total elapsed time: 00h 00m 55s
INFO:tensorflow:Oracle triggered exit


##Выбираем 3 лучших модели

In [9]:
tuner.results_summary(3)

Results summary
Results in test_directory/untitled_project
Showing 3 best trials
Trial summary
Hyperparameters:
units_input: 896
Score: 0.7918333411216736
Trial summary
Hyperparameters:
units_input: 672
Score: 0.7872499823570251
Trial summary
Hyperparameters:
units_input: 768
Score: 0.7857499718666077


Получаем три лучших модели

In [10]:
models = tuner.get_best_models(num_models=3)

Оцениваем качество выбранных моделей на тестовых данных

In [11]:
for model in models:
  model.summary()
  model.evaluate(x_test, y_test)
  print()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 896)               703360    
                                                                 
 dense_1 (Dense)             (None, 10)                8970      
                                                                 
Total params: 712,330
Trainable params: 712,330
Non-trainable params: 0
_________________________________________________________________
313/313 [==============================] - 1s 3ms/step - loss: 0.6848 - accuracy: 0.7785

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 672)               527520    
                                                                 
 dense_1 (Dense)             (None, 10)                6730      
      

##Улучшим функцию подбора гиперпараметров

Добавим один скрытый слой

In [12]:
def build_model_1(hp):
    model = Sequential() 
    model.add(Dense(units=hp.Int('units_input',    # Полносвязный слой с разным количеством нейронов
                                   min_value=512,    # минимальное количество нейронов - 512
                                   max_value=1024,   # максимальное количество - 1024
                                   step=32),
                    input_dim=784,
                    activation='relu'))
    model.add(Dense(units=hp.Int('units_hidden',        
                                   min_value=128,   
                                   max_value=600,   
                                   step=32),
                    activation='relu'))   
    model.add(Dense(10, activation='softmax'))
    model.compile(
        optimizer='SGD',
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

##Создаём тюнер

## Задаем функцию создания нейронной сети

In [13]:
tuner = BayesianOptimization(
    build_model_1,
    objective='val_accuracy',
    max_trials=10,
    directory='test_directory_1'
)

##Запускаем подбор гиперпараметров

Пространство поиска

In [14]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 512, 'max_value': 1024, 'step': 32, 'sampling': None}
units_hidden (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 600, 'step': 32, 'sampling': None}


Подбор гиперпараметров

In [15]:
tuner.search(
    x_train,
    y_train,
    batch_size=256,
    epochs=3,
    validation_split=0.2,
    verbose=1,
)

Trial 10 Complete [00h 00m 04s]
val_accuracy: 0.79666668176651

Best val_accuracy So Far: 0.8011666536331177
Total elapsed time: 00h 01m 04s
INFO:tensorflow:Oracle triggered exit


Получим три лучших модели

In [16]:
models_1 = tuner.get_best_models(num_models=3)

Оцениваем качество модели на тестовых данных

In [17]:
for model_1 in models_1:
  model.summary()
  model.evaluate(x_test, y_test)
  print

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 768)               602880    
                                                                 
 dense_1 (Dense)             (None, 10)                7690      
                                                                 
Total params: 610,570
Trainable params: 610,570
Non-trainable params: 0
_________________________________________________________________
313/313 [==============================] - 1s 3ms/step - loss: 0.6980 - accuracy: 0.7747
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 768)               602880    
                                                                 
 dense_1 (Dense)             (None, 10)                7690      
       

##Задаём функцию нейронной сети

In [18]:
def build_model_2(hp):
    model = Sequential()
    activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])    
    model.add(Dense(units=hp.Int('units_input',    # Полносвязный слой с разным количеством нейронов
                                   min_value=512,    # минимальное количество нейронов - 128
                                   max_value=1024,   # максимальное количество - 1024
                                   step=32),
                    input_dim=784,
                    activation=activation_choice))
    model.add(Dense(units=hp.Int('units_hidden',        
                                   min_value=128,   
                                   max_value=600,   
                                   step=32),
                    activation=activation_choice))   
    model.add(Dense(10, activation='softmax'))
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam','rmsprop','SGD']),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

## Создаем tuner

Доступные типы тюнеров: 
- RandomSearch - случайный поиск.
- Hyperband - алгоритм оптимизации на основе многорукого бандита, Li, Lisha, and Kevin Jamieson. ["Hyperband: A Novel Bandit-Based Approach to Hyperparameter Optimization."Journal of Machine Learning Research 18 (2018): 1-52](http://jmlr.org/papers/v18/16-558.html).
- BayesianOptimization - [байесовская оптимизация](https://en.wikipedia.org/wiki/Bayesian_optimization).

In [19]:
tuner = BayesianOptimization(
    build_model_2,                 # функция создания модели
    objective='val_accuracy',    # метрика, которую нужно оптимизировать - 
                                 # доля правильных ответов на проверочном наборе данных
    max_trials=80,               # максимальное количество запусков обучения 
    directory='test_directory_2'   # каталог, куда сохраняются обученные сети  
    )

## Запускаем подбор гиперпараметров

Пространство поиска

In [20]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh', 'elu', 'selu'], 'ordered': False}
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 512, 'max_value': 1024, 'step': 32, 'sampling': None}
units_hidden (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 600, 'step': 32, 'sampling': None}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'rmsprop', 'SGD'], 'ordered': False}


Подбор гиперпараметров

In [21]:
tuner.search(x_train,                  # Данные для обучения
             y_train,                  # Правильные ответы
             batch_size=256,           # Размер мини-выборки
             epochs=3,                # Количество эпох обучения 
             validation_split=0.2,     # Часть данных, которая будет использоваться для проверки
             )

Trial 80 Complete [00h 00m 05s]
val_accuracy: 0.8812500238418579

Best val_accuracy So Far: 0.8829166889190674
Total elapsed time: 00h 13m 02s
INFO:tensorflow:Oracle triggered exit


## Выбираем лучшую модель

In [22]:
tuner.results_summary()

Results summary
Results in test_directory_2/untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
activation: relu
units_input: 1024
units_hidden: 128
optimizer: adam
Score: 0.8829166889190674
Trial summary
Hyperparameters:
activation: relu
units_input: 1024
units_hidden: 128
optimizer: adam
Score: 0.8812500238418579
Trial summary
Hyperparameters:
activation: relu
units_input: 1024
units_hidden: 128
optimizer: adam
Score: 0.8799166679382324
Trial summary
Hyperparameters:
activation: relu
units_input: 1024
units_hidden: 128
optimizer: adam
Score: 0.8796666860580444
Trial summary
Hyperparameters:
activation: relu
units_input: 1024
units_hidden: 128
optimizer: adam
Score: 0.8794166445732117
Trial summary
Hyperparameters:
activation: relu
units_input: 1024
units_hidden: 128
optimizer: adam
Score: 0.8790833353996277
Trial summary
Hyperparameters:
activation: relu
units_input: 1024
units_hidden: 128
optimizer: adam
Score: 0.8790000081062317
Trial summary
Hyperparameters:
act

Получаем три лучших модели

In [23]:
models_2 = tuner.get_best_models(num_models=3)

Оцениваем качество модели на тестовых данных

In [24]:
for model_2 in models_2:
  model.summary()
  model.evaluate(x_test, y_test)
  print() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 768)               602880    
                                                                 
 dense_1 (Dense)             (None, 10)                7690      
                                                                 
Total params: 610,570
Trainable params: 610,570
Non-trainable params: 0
_________________________________________________________________
313/313 [==============================] - 1s 3ms/step - loss: 0.6980 - accuracy: 0.7747

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 768)               602880    
                                                                 
 dense_1 (Dense)             (None, 10)                7690      
      

##Сделаем количество слоёв гиперпараметром

In [27]:
def build_model_3(hp):
    model = Sequential()
    activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])    
    model.add(Dense(units=hp.Int('units_input',    # Полносвязный слой с разным количеством нейронов
                                   min_value=512,    # минимальное количество нейронов - 128
                                   max_value=1024,   # максимальное количество - 1024
                                   step=32),
                    input_dim=784,
                    activation=activation_choice))
    for i in range(hp.Int('num_layers', 2, 5)):
      model.add(Dense(units=hp.Int('units_' + str(i),
                                          min_value=128,
                                          max_value=1024,
                                          step=32),
                             activation='relu'))  
    model.add(Dense(10, activation='softmax'))
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam','rmsprop','SGD']),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

## Создаем тюнер

In [28]:
tuner = BayesianOptimization(
    build_model_3,                 # функция создания модели
    objective='val_accuracy',    # метрика, которую нужно оптимизировать - 
                                 # доля правильных ответов на проверочном наборе данных
    max_trials=40,               # максимальное количество запусков обучения 
    directory='test_directory_3'   # каталог, куда сохраняются обученные сети  
    )

## Запускаем подбор гиперпараметров

Пространство поиска

In [29]:
tuner.search_space_summary()

Search space summary
Default search space size: 6
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh', 'elu', 'selu'], 'ordered': False}
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 512, 'max_value': 1024, 'step': 32, 'sampling': None}
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 5, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 1024, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 1024, 'step': 32, 'sampling': None}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'rmsprop', 'SGD'], 'ordered': False}


Подбор гиперпараметров

In [30]:
tuner.search(x_train,                  # Данные для обучения
             y_train,                  # Правильные ответы
             batch_size=256,           # Размер мини-выборки
             epochs=3,                # Количество эпох обучения 
             validation_split=0.2,     # Часть данных, которая будет использоваться для проверки
             )

Trial 40 Complete [00h 00m 06s]
val_accuracy: 0.8788333535194397

Best val_accuracy So Far: 0.8813333511352539
Total elapsed time: 00h 10m 02s
INFO:tensorflow:Oracle triggered exit


## Выбираем лучшую модель

In [31]:
tuner.results_summary()

Results summary
Results in test_directory_3/untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
activation: selu
units_input: 960
num_layers: 2
units_0: 1024
units_1: 288
optimizer: adam
units_2: 1024
units_3: 192
units_4: 128
Score: 0.8813333511352539
Trial summary
Hyperparameters:
activation: selu
units_input: 544
num_layers: 2
units_0: 800
units_1: 416
optimizer: adam
units_2: 320
Score: 0.8804166913032532
Trial summary
Hyperparameters:
activation: selu
units_input: 512
num_layers: 2
units_0: 1024
units_1: 1024
optimizer: adam
units_2: 1024
Score: 0.8794999718666077
Trial summary
Hyperparameters:
activation: selu
units_input: 512
num_layers: 2
units_0: 1024
units_1: 896
optimizer: adam
units_2: 448
units_3: 1024
units_4: 832
Score: 0.8790000081062317
Trial summary
Hyperparameters:
activation: selu
units_input: 512
num_layers: 2
units_0: 1024
units_1: 800
optimizer: adam
units_2: 128
units_3: 128
units_4: 128
Score: 0.8788333535194397
Trial summary
Hyperparameters:

Получаем три лучших модели

In [34]:
models_3 = tuner.get_best_models(num_models=3)

Оцениваем качество модели на тестовых данных

In [36]:
for model_3 in models_3:
  model_3.summary()
  model_3.evaluate(x_test, y_test)
  print() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 960)               753600    
                                                                 
 dense_1 (Dense)             (None, 1024)              984064    
                                                                 
 dense_2 (Dense)             (None, 288)               295200    
                                                                 
 dense_3 (Dense)             (None, 10)                2890      
                                                                 
Total params: 2,035,754
Trainable params: 2,035,754
Non-trainable params: 0
_________________________________________________________________
313/313 [==============================] - 1s 4ms/step - loss: 0.3541 - accuracy: 0.8680

Model: "sequential"
_________________________________________________________________
 L